<a href="https://colab.research.google.com/github/aruay99/Sudoku/blob/main/Sudoku%20Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Discrete Optimization - Sudoku Solver

In [ ]:
import time
!pip install pulp
from pulp import *

## Naive Approach
Generate all possible configurations of numbers from 1 to 9 to fill the empty cells. Try every configuration one by one until the correct configuration is found, i.e. for every unassigned position fill the position with a number from 1 to 9. After filling all the unassigned position check if the matrix is safe or not. If safe print else recurs for other cases.*italicised text*

In [ ]:
# code adapted from source: https://www.geeksforgeeks.org/sudoku-backtracking-7/

# A utility function to print grid
def printing(sudoku):
    for i in range(len(sudoku)):
        line = ""
        if i == 3 or i == 6:
            print("---------------------")
        for j in range(len(sudoku[i])):
            if j == 3 or j == 6:
                line += "| "
            line += str(sudoku[i][j])+" "
        print(line)
 
# Checks whether it will be
# legal to assign num to the
# given row, col
def isSafe(grid, row, col, num):
   
    # Check if we find the same num
    # in the similar row , we
    # return false
    for x in range(9):
        if grid[row][x] == num:
            return False
 
    # Check if we find the same num in
    # the similar column , we
    # return false
    for x in range(9):
        if grid[x][col] == num:
            return False
 
    # Check if we find the same num in
    # the particular 3*3 matrix,
    # we return false
    startRow = row - row % 3
    startCol = col - col % 3
    for i in range(3):
        for j in range(3):
            if grid[i + startRow][j + startCol] == num:
                return False
    return True
 
# Takes a partially filled-in grid and attempts
# to assign values to all unassigned locations in
# such a way to meet the requirements for
# Sudoku solution (non-duplication across rows,
# columns, and boxes) */
def solveSuduko(grid, row, col):
   
    # Check if we have reached the 8th
    # row and 9th column (0
    # indexed matrix) , we are
    # returning true to avoid
    # further backtracking
    if (row == 9 - 1 and col == 9):
        return True
       
    # Check if column value  becomes 9 ,
    # we move to next row and
    # column start from 0
    if col == 9:
        row += 1
        col = 0
 
    # Check if the current position of
    # the grid already contains
    # value >0, we iterate for next column
    if grid[row][col] > 0:
        return solveSuduko(grid, row, col + 1)
    for num in range(1, 9 + 1, 1):
       
        # Check if it is safe to place
        # the num (1-9)  in the
        # given row ,col  ->we
        # move to next column
        if isSafe(grid, row, col, num):
           
            # Assigning the num in
            # the current (row,col)
            # position of the grid
            # and assuming our assined
            # num in the position
            # is correct
            grid[row][col] = num
 
            # Checking for next possibility with next
            # column
            if solveSuduko(grid, row, col + 1):
                return True
 
        # Removing the assigned num ,
        # since our assumption
        # was wrong , and we go for
        # next assumption with
        # diff num value
        grid[row][col] = 0
    return False
 


if (solveSuduko(grid, 0, 0)):
    printing(grid)
    print("--- %s seconds ---" % (time.time() - start_time))
else:
    print("no solution  exists ")


no solution  exists 


## Backtracking Approach



In [ ]:
# code adapted from source: https://www.techwithtim.net/tutorials/python-programming/sudoku-solver-backtracking/
def backtrackingSolver(bo):
    sol = bo
    find = find_empty(bo)
    if not find:
        return sol, True
    else:
        row, col = find

    for i in range(1,10):
        if valid(bo, i, (row, col)):
            sol[row][col] = i
            if solve(bo):
                return sol, True
            sol[row][col] = 0
    return sol, False


def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True


def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col
    return None

## Linear Programming

In [ ]:
def linearProgrammingSolver(grid):
  number = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
  vals = number
  rows = number
  cols = number

  subGrids =[]
  for i in range(3):
      for j in range(3):
          subGrids += [[(rows[3*i+k],cols[3*j+l]) for k in range(3) for l in range(3)]]
  
  #Create the LP problem
  prob = LpProblem("Sudoku Problem",LpMinimize)

  #Set the decision variables
  choices = LpVariable.dicts("Choice",(vals,rows,cols),0,1,LpInteger)

  #Add main objective function
  prob += 0, "Arbitrary Objective Function"

  #Add constraints:
  for r in rows:
      for c in cols:
          prob += lpSum([choices[v][r][c] for v in vals]) == 1, ""

  for v in vals:
      for r in rows:
          prob += lpSum([choices[v][r][c] for c in cols]) == 1,""
          
      for c in cols:
          prob += lpSum([choices[v][r][c] for r in rows]) == 1,""

      for b in subGrids:
          prob += lpSum([choices[v][r][c] for (r,c) in b]) == 1,""
  
  for curR in number:
    for curC in number:
      v = grid[int(curR)-1][int(curC)-1]
      if v != 0:
        prob += choices[str(v)][curR][curC] == 1,""

  prob.writeLP("Sudoku.lp")
  prob.solve()

  sol = [[ 0 for i in range(9) ] for j in range(9) ]
  for r in rows:
      for c in cols:
          for v in vals:
              if value(choices[v][r][c])==1:     
                  sol[int(r)-1][int(c)-1] = v
  return sol


In [ ]:
easy = [[7,0,0,4,2,0,9,0,1],
       [2,0,0,3,1,9,0,5,7],
       [0,9,3,7,5,6,8,0,4],
       [9,5,8,2,0,0,7,0,0],
       [4,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,2,0,8],
       [5,4,6,1,0,7,0,9,0],
       [3,7,0,0,9,0,0,8,5],
       [8,0,0,5,4,0,0,0,0]]

medium = [[0,4,0,0,0,2,0,1,9],
       [0,0,0,3,5,1,0,8,6],
       [3,1,0,0,9,4,7,0,0],
       [0,9,4,0,0,0,0,0,7],
       [0,0,0,0,0,0,0,0,0],
       [2,0,0,0,0,0,8,9,0],
       [0,0,9,5,2,0,0,4,1],
       [4,2,0,1,6,9,0,0,0],
       [1,6,0,8,0,0,0,7,0]]

insanelyHard = [[0,0,5,3,0,0,0,0,0],
       [8,0,0,0,0,0,0,2,0],
       [0,7,0,0,1,0,5,0,0],
       [4,0,0,0,0,5,3,0,0],
       [0,1,0,0,7,0,0,0,6],
       [0,0,3,2,0,0,0,8,0],
       [0,6,0,0,0,0,2,8,0],
       [0,0,4,0,0,0,0,3,0],
       [0,0,0,0,0,9,7,0,0]]

hard = [[0,0,5,3,0,0,0,0,0],
       [8,0,0,0,0,0,0,2,0],
       [0,7,0,0,1,0,5,0,0],
       [4,0,0,0,0,5,3,0,0],
       [0,1,0,0,7,0,0,0,6],
       [0,0,3,2,0,0,0,8,0],
       [0,6,0,5,0,0,0,0,9],
       [0,0,4,0,0,0,0,3,0],
       [0,0,0,0,0,9,7,0,0]]

allLevels = [easy, medium, hard, insanelyHard]

for level in allLevels:
  print("Current: ")
  print()
  printing(level)
  print()

  #backtracking
  print("Backtracking: ")
  print()
  start_time = time.time()
  sol, solved = backtrackingSolver(level)
  if solved:
    printing(sol)
  else:
    print("no solution was found")
  print("--- %s seconds ---" % (time.time() - start_time))
  print()

  #linear programmming
  print("Linear Programming: ")
  print()
  start_time = time.time()
  printing(linearProgrammingSolver(level))
  print("--- %s seconds ---" % (time.time() - start_time))
  print()



Current: 

7 0 0 | 4 2 0 | 9 0 1 
2 0 0 | 3 1 9 | 0 5 7 
0 9 3 | 7 5 6 | 8 0 4 
---------------------
9 5 8 | 2 0 0 | 7 0 0 
4 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 2 0 8 
---------------------
5 4 6 | 1 0 7 | 0 9 0 
3 7 0 | 0 9 0 | 0 8 5 
8 0 0 | 5 4 0 | 0 0 0 

Backtracking: 

7 6 5 | 4 2 8 | 9 3 1 
2 8 4 | 3 1 9 | 6 5 7 
1 9 3 | 7 5 6 | 8 2 4 
---------------------
9 5 8 | 2 6 4 | 7 1 3 
4 3 2 | 8 7 1 | 5 6 9 
6 1 7 | 9 3 5 | 2 4 8 
---------------------
5 4 6 | 1 8 7 | 3 9 2 
3 7 1 | 6 9 2 | 4 8 5 
8 2 9 | 5 4 3 | 1 7 6 
--- 0.017908096313476562 seconds ---

Linear Programming: 

7 6 5 | 4 2 8 | 9 3 1 
2 8 4 | 3 1 9 | 6 5 7 
1 9 3 | 7 5 6 | 8 2 4 
---------------------
9 5 8 | 2 6 4 | 7 1 3 
4 3 2 | 8 7 1 | 5 6 9 
6 1 7 | 9 3 5 | 2 4 8 
---------------------
5 4 6 | 1 8 7 | 3 9 2 
3 7 1 | 6 9 2 | 4 8 5 
8 2 9 | 5 4 3 | 1 7 6 
--- 0.08557295799255371 seconds ---

Current: 

0 4 0 | 0 0 2 | 0 1 9 
0 0 0 | 3 5 1 | 0 8 6 
3 1 0 | 0 9 4 | 7 0 0 
---------------------
0 9 4 | 0 0 0 | 0 0 

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")



7 8 3 | 9 1 5 | 2 6 4 
2 5 1 | 4 7 6 | 8 9 3 
---------------------
8 3 9 | 5 2 7 | 6 4 1 
4 2 7 | 1 6 9 | 5 3 8 
1 6 5 | 8 4 3 | 9 7 2 
--- 0.09047842025756836 seconds ---

Current: 

0 0 5 | 3 0 0 | 0 0 0 
8 0 0 | 0 0 0 | 0 2 0 
0 7 0 | 0 1 0 | 5 0 0 
---------------------
4 0 0 | 0 0 5 | 3 0 0 
0 1 0 | 0 7 0 | 0 0 6 
0 0 3 | 2 0 0 | 0 8 0 
---------------------
0 6 0 | 5 0 0 | 0 0 9 
0 0 4 | 0 0 0 | 0 3 0 
0 0 0 | 0 0 9 | 7 0 0 

Backtracking: 

1 4 5 | 3 2 7 | 6 9 8 
8 3 9 | 6 5 4 | 1 2 7 
6 7 2 | 9 1 8 | 5 4 3 
---------------------
4 9 6 | 1 8 5 | 3 7 2 
2 1 8 | 4 7 3 | 9 5 6 
7 5 3 | 2 9 6 | 4 8 1 
---------------------
3 6 7 | 5 4 2 | 8 1 9 
9 8 4 | 7 6 1 | 2 3 5 
5 2 1 | 8 3 9 | 7 6 4 
--- 0.21696972846984863 seconds ---

Linear Programming: 

1 4 5 | 3 2 7 | 6 9 8 
8 3 9 | 6 5 4 | 1 2 7 
6 7 2 | 9 1 8 | 5 4 3 
---------------------
4 9 6 | 1 8 5 | 3 7 2 
2 1 8 | 4 7 3 | 9 5 6 
7 5 3 | 2 9 6 | 4 8 1 
---------------------
3 6 7 | 5 4 2 | 8 1 9 
9 8 4 | 7 6 1 | 2 3 5 
5 2 1 | 